In [68]:
!pip install transformers

In [69]:
import os

import random
import numpy as np

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import (
    AdamW,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import gc

from transformers import pipeline
import re

In [70]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


# Load the training data
data = pd.read_csv('/kaggle/input/murderdata/Mysterious_Affair_at_Styles_Train_Data.csv')


In [71]:
data

,OrginalPage,Text
0,1,CHAPTER I.\nI GO TO STYLES\n\n\nThe intense in...
1,2,"looked his forty-five years. As a boy, though,..."
2,3,at the\ntime of their father's remarriage that...
3,4,"black beard,\nand wears patent leather boots i..."
4,5,"""I'm afraid you'll find it very quiet down her..."
...,...,...
202,203,hers! It concealed a very opposite emotion.\nT...
203,204,them to us. There was really more\nevidence ag...
204,205,something about the matter. He was nearly\ndri...
205,206,"""Yes, I quite understood your distress.""\n\n""P..."


In [72]:
split_data = pd.DataFrame(columns=['text', 'OrginalPage'])

In [73]:
data

,OrginalPage,Text
0,1,CHAPTER I.\nI GO TO STYLES\n\n\nThe intense in...
1,2,"looked his forty-five years. As a boy, though,..."
2,3,at the\ntime of their father's remarriage that...
3,4,"black beard,\nand wears patent leather boots i..."
4,5,"""I'm afraid you'll find it very quiet down her..."
...,...,...
202,203,hers! It concealed a very opposite emotion.\nT...
203,204,them to us. There was really more\nevidence ag...
204,205,something about the matter. He was nearly\ndri...
205,206,"""Yes, I quite understood your distress.""\n\n""P..."


In [74]:
# for _, row in data.iterrows():
#     # Split the text into two equal parts
#     text1, text2 = row['Text'][:len(row['Text']) // 2], row['Text'][len(row['Text']) // 2:]
#     # Add the split texts to the dataframe as separate rows
#     split_data = split_data.append({'text': text1, 'OrginalPage': row['OrginalPage']}, ignore_index=True)
#     split_data = split_data.append({'text': text2, 'OrginalPage': row['OrginalPage']}, ignore_index=True)
    
# split_data=split_data.sample(frac=1)
# # Print the resulting dataframe
# split_data

In [75]:
def remove_newlines(text):
    return re.sub(r'\n', '', text)

In [76]:
# Preprocess the text data by lowercasing and removing punctuation
# split_data['text'] = split_data['text'].apply(remove_newlines)
data['Text'] = data['Text'].apply(remove_newlines)

In [77]:
# split_data

In [78]:
# X = split_data['text']
# X = X.str.lower().str.replace(r'[^\w\s]', '')
# y = split_data['OrginalPage']
X = data['Text']
X = X.str.lower().str.replace(r'[^\w\s]', '')
y = data['OrginalPage']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [79]:
vectorizer = CountVectorizer()
tfidf_dataa = vectorizer.fit_transform(X).toarray()
features = vectorizer.get_feature_names_out()
tfidf_data = pd.DataFrame(tfidf_dataa, columns=features)

In [80]:
from sklearn.decomposition import LatentDirichletAllocation

# Create a LatentDirichletAllocation object
lda = LatentDirichletAllocation(n_components=12)

# Fit the model to the data and transform the text to a topic representation
X_topics = lda.fit_transform(tfidf_dataa)
tfidf_data = pd.concat([tfidf_data, pd.DataFrame(X_topics)], axis=1)

In [81]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest=train_test_split(tfidf_data, y, test_size=0.1, random_state=11)

In [82]:
# !pip install lazypredict

In [83]:
# from lazypredict.Supervised import LazyRegressor
# from sklearn.utils import shuffle
# import numpy as np

# reg = LazyRegressor(verbose=0,ignore_warnings=False, custom_metric=None )
# models,predictions = reg.fit(xtrain, xtest, ytrain, ytest)

In [84]:
# models

In [85]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.linear_model import QuantileRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier , StackingRegressor, VotingRegressor
from sklearn.linear_model import OrthogonalMatchingPursuitCV, ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import GammaRegressor

model4=GammaRegressor()

model3 = HuberRegressor()
model2 = ElasticNetCV()
model1 = ElasticNet()
voting_model = StackingRegressor(estimators=[('model_1', model1), ('model_2', model2)
                                            , ('model_3', model3)])


# Make predictions on the test data

reg = voting_model.fit(tfidf_data, y)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_huber.py:332: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/

In [86]:
# from sklearn.linear_model import OrthogonalMatchingPursuitCV
# reg = OrthogonalMatchingPursuitCV(cv=5).fit(tfidf_data, y)

In [87]:
# Load the test data
test_data = pd.read_csv('/kaggle/input/murderdata/Cain_s_Jawbone_Test_data.csv')
test_data['Text'] = test_data['Text'].apply(remove_newlines)
X_test = test_data['Text']
# Preprocess the test data in the same way as the training data
X_test = X_test.str.lower().str.replace(r'[^\w\s]', '')
# Extract features from the test data using the same vectorizer


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [88]:
testt = vectorizer.transform(X_test).toarray()
features = vectorizer.get_feature_names_out()
test_data = pd.DataFrame(testt, columns=features)

X_topicss = lda.transform(testt)
test_data = pd.concat([test_data, pd.DataFrame(X_topicss)], axis=1)

In [89]:
y__pred=reg.predict(test_data)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [90]:
y__pred

array([89.00936967, 88.4532301 , 97.16646914, 91.58970499, 89.47903147,
       87.72846089, 86.13484452, 89.0660577 , 90.69391759, 92.65548608,
       92.77911401, 86.67214428, 86.49993286, 91.91785662, 90.60885365,
       88.59766189, 89.2199662 , 90.12186377, 85.58838996, 88.36792459,
       85.30341989, 91.84038447, 91.29256965, 95.6138898 , 89.85596023,
       86.33823843, 90.72941831, 90.63899452, 86.93356356, 87.21009709,
       82.95216953, 86.61732104, 96.55327895, 95.88862679, 85.7121128 ,
       89.47939774, 87.96940025, 89.57606921, 88.81393407, 91.07570164,
       85.54533614, 84.66499341, 86.00145289, 87.11140335, 85.39863913,
       91.76070243, 87.67722177, 92.2776035 , 85.727519  , 89.61975739,
       88.41876791, 92.27607099, 87.42320144, 89.10388403, 86.46130362,
       89.58829316, 88.55618769, 88.88695353, 90.60362443, 86.20128256,
       88.21389481, 88.91630512, 92.26170367, 87.62768231, 88.55304579,
       87.93712874, 96.01947494, 89.45714809, 86.71754667, 88.36

In [91]:
pred=[round((i)) for i in y__pred]

In [92]:
predd=[int((i)) for i in y__pred]

In [93]:
len(set(pred)) , len(set(predd))

(12, 14)

In [94]:
set(pred), set(predd)

({83, 85, 86, 87, 88, 89, 90, 91, 92, 93, 96, 97},
 {82, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97})

In [95]:
sub=pd.read_csv("/kaggle/input/murderdata/SampleSubmission.csv")

In [108]:
sub["CorrectPage"] = predd

In [109]:
sub.CorrectPage

0     89
1     88
2     97
3     91
4     89
      ..
70    88
71    87
72    93
73    95
74    91
Name: CorrectPage, Length: 75, dtype: int64

In [110]:
sub.CorrectPage.nunique()

14

In [113]:
sub.to_csv("sub24.csv", index=False)

In [112]:
sub

,OriginalPage,CorrectPage
0,1,89
1,2,88
2,3,97
3,4,91
4,5,89
...,...,...
70,71,88
71,72,87
72,73,93
73,74,95
